In [1]:
import os

In [2]:
%pwd

'/Users/shlok/DSS/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/shlok/DSS/Text-Summarizer'

In [5]:
#Entity(ReturnType of functions)

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig: #This is a dataclass not a python class
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)


    def get_data_ingestion_config(self)->DataIngestionConfig:

        config=self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with info:\n{headers}")

        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))}")   

            
    def extract_zip_file (self):
        """
        Zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs (unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [37]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2023-06-06 10:39:42,754]: INFO:common:yaml file: config/config.yaml loaded successfully
[2023-06-06 10:39:42,756]: INFO:common:yaml file: params.yaml loaded successfully
[2023-06-06 10:39:42,756]: INFO:common:created directory at: artifacts/data_ingestion
[2023-06-06 10:39:44,701]: INFO:1842121531:artifacts/data_ingestion/data.zip downloaded with info:
Connection: close
Content-Length: 4140214
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "eeb785dd47862f3a4c526963760a4e4d1f4739defb4cf02f30a9836b30a2edf9"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 5D74:24ED:30A9F:52E4B:647EBF97
Accept-Ranges: bytes
Date: Tue, 06 Jun 2023 05:09:44 GMT
Via: 1.1 varnish
X-Served-By: cache-del21739-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1686028184.581963,VS0,VE571
Vary: Authorization,Accept-Encoding,